In [1]:
## PREREQUISITE FOR RUNNUNG ON GOOGLE COLAB

from urllib.request import urlretrieve
from zipfile import ZipFile
import os

# !rm -rf CarND-Behavioral-Cloning-P3/

if ("CarND-Behavioral-Cloning-P3" not in os.listdir(".")):
  !git clone https://github.com/udacity/CarND-Behavioral-Cloning-P3.git
  !pip install opencv-python
  !apt update && apt install -y libsm6 libxext6
  !pip install -q keras
  urlretrieve("https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip","./CarND-Behavioral-Cloning-P3/data.zip")
  with ZipFile("./CarND-Behavioral-Cloning-P3/data.zip") as zipf:
    zipf.extractall("./CarND-Behavioral-Cloning-P3/")
  !pip install -q imageio       
  !pip install moviepy
  !python -c "exec(\"import imageio\\nimageio.plugins.ffmpeg.download()\")"
  !pip install flask-socketio
  !pip install eventlet
  !apt-get -o Dpkg::Options::="--force-confmiss" install --reinstall netbase

Cloning into 'CarND-Behavioral-Cloning-P3'...
remote: Counting objects: 86, done.
remote: Total 86 (delta 0), reused 0 (delta 0), pack-reused 86
Unpacking objects: 100% (86/86), done.
    100% |████████████████████████████████| 24.9MB 67kB/s 
Hit:1 http://archive.ubuntu.com/ubuntu artful InRelease
Get:2 http://security.ubuntu.com/ubuntu artful-security InRelease [78.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu artful-updates InRelease [78.6 kB]
Get:4 http://security.ubuntu.com/ubuntu artful-security/universe Sources [12.0 kB]
Get:5 http://archive.ubuntu.com/ubuntu artful-backports InRelease [72.2 kB]
Get:6 http://security.ubuntu.com/ubuntu artful-security/universe amd64 Packages [55.0 kB]
Get:7 http://security.ubuntu.com/ubuntu artful-security/main amd64 Packages [171 kB]
Get:8 http://archive.ubuntu.com/ubuntu artful-updates/universe Sources [29.3 kB]
Get:9 http://archive.ubuntu.com/ubuntu artful-updates/universe amd64 Packages [110 kB]
Get:10 http://archive.ubuntu.com/ubuntu artful-up

invoke-rc.d: policy-rc.d denied execution of start.
7Progress: [ 76%] [############################################..............] 8Setting up libice6:amd64 (2:1.0.9-2) ...
7Progress: [ 80%] [##############################################............] 87Progress: [ 85%] [#################################################.........] 8Setting up libsm6:amd64 (2:1.2.2-1) ...
7Progress: [ 90%] [####################################################......] 87Progress: [ 95%] [#######################################################...] 8Processing triggers for libc-bin (2.26-0ubuntu2.1) ...

78Collecting moviepy
    100% |████████████████████████████████| 133kB 3.1MB/s 
    100% |████████████████████████████████| 51kB 5.1MB/s 
    100% |████████████████████████████████| 3.3MB 462kB/s 
  Running setup.py bdist_wheel for imageio ... - \ | done
  Stored in directory: /content/.cache/pip/wheels/47/a7/c8/2d9d42949793f2ad22a9f7974be42b34f77b97d28aa206cf94
Successfully built imageio
 

    100% |████████████████████████████████| 71kB 7.2MB/s 
    100% |████████████████████████████████| 51kB 8.8MB/s 
  Running setup.py bdist_wheel for itsdangerous ... - done
  Stored in directory: /content/.cache/pip/wheels/fc/a8/66/24d655233c757e178d45dea2de22a04c6d92766abfb741129a
Successfully built itsdangerous
    100% |████████████████████████████████| 409kB 2.2MB/s 
    100% |████████████████████████████████| 51kB 7.4MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  netbase
0 upgraded, 1 newly installed, 0 to remove and 36 not upgraded.
Need to get 12.7 kB of archives.
After this operation, 45.1 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu artful/main amd64 netbase all 5.4 [12.7 kB]
Fetched 12.7 kB in 0s (71.6 kB/s)
Selecting previously unselected package netbase.
(Reading database ... 16712 files and directories currently installed.)
Preparin

In [0]:
import csv
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import gc
import random


class DataSet:
    lines_train = None
    lines_test = None

    gen_train = None
    gen_test = None

    train_size = None
    test_size = None

    image_shape = None

    base_path = None
    batch_size = None

    def __init__(self, data_file, batch_size=64):
        self.batch_size = batch_size
        self.base_path = data_file.rsplit("/", 1)[0]
        lines = []
        with open(data_file) as csv_file:
            reader = csv.reader(csv_file)
            for line in reader:
                lines.append(line)
        lines = lines[1:]
        self.lines_train, self.lines_test = train_test_split(lines, test_size=0.2)
        self.train_size = len(self.lines_train)
        self.test_size = len(self.lines_test)
        self.n = len(lines)
        self.image_shape = cv2.imread(self.base_path + "/" + lines[0][0].strip()).shape


    def __generate(self, lines):
        while True:
            shuffle(lines)
            for offset in range(0, len(lines), self.batch_size):
                batch = lines[offset: offset + self.batch_size]
                
                views = []
                steers = []
                for line in batch:
                  
                    view = [cv2.imread(self.base_path + "/" + (line[i].strip())) for i in range(3)]
                    steer = [float(line[3]),0.25+float(line[3]),float(line[3])-0.25]
                    
                    for img in list(view):
                      view.append(np.fliplr(img))
                      
                    for angle in list(steer):
                      steer.append(-angle)
                      
                    index = np.zeros(len(view))
                    if len(views) > 0:
                        index = [random.randrange(0, len(views)+i) for i in range(len(index))]  
                        
                    for i in range(len(index)):
                      views.insert(int(index[i]), view[i])
                      steers.insert(int(index[i]), steer[i])
                      
                
                views = views[:len(lines)]
                steers = steers[:len(lines)]             

                X = np.array(views)
                y = np.array(steers)
                gc.collect()
                yield X, y

    def for_training(self):
        if self.gen_train is None:
            self.gen_train = self.__generate(self.lines_train)
        return self.gen_train

    def for_testing(self):
        if self.gen_test is None:
            self.gen_test = self.__generate(self.lines_test)
        return self.gen_test

In [60]:
dataset = DataSet("./CarND-Behavioral-Cloning-P3/data/driving_log.csv")

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Lambda, Dropout, Cropping2D, Activation, Merge, Input, ELU
from keras.layers.convolutional import Convolution2D
from keras.layers.core import *
from keras.models import Model
from keras.layers.merge import concatenate

def nvidia_functional():
    imagein = Input(shape=dataset.image_shape)
    layer = Cropping2D(cropping=((70, 20), (0, 0)))(imagein)
    layer = Lambda(lambda x: (x / 127.5) - 1)(layer)
    layer = Convolution2D(24, 5, 5, border_mode='valid', activation='relu', subsample=(2, 2))(layer)
    layer = Convolution2D(36, 5, 5, border_mode='valid', activation='relu', subsample=(2, 2))(layer)
    layer = Convolution2D(48, 5, 5, border_mode='valid', activation='relu', subsample=(2, 2))(layer)
    layer = Convolution2D(64, 3, 3, border_mode='valid', activation='relu', subsample=(1, 1))(layer)
    layer = Convolution2D(64, 3, 3, border_mode='valid', activation='relu', subsample=(1, 1))(layer)
    layer = Flatten()(layer)
    layer = Dense(100, activation="elu")(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(50, activation="elu")(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(10)(layer)
    steerout = Dense(1)(layer)
    model = Model(inputs=imagein, outputs=steerout)
    return model

def nvidia_sequential():
    model = Sequential()
    model.add(Cropping2D(cropping=((70, 25), (0, 0)), input_shape=dataset.image_shape))
    model.add(Lambda(lambda x: (x / 127.5) - 1))
    model.add(Convolution2D(24, 5, 5, border_mode='valid', activation='relu', subsample=(2, 2)))
    model.add(Convolution2D(36, 5, 5, border_mode='valid', activation='relu', subsample=(2, 2)))
    model.add(Convolution2D(48, 5, 5, border_mode='valid', activation='relu', subsample=(2, 2)))
    model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu', subsample=(1, 1)))
    model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu', subsample=(1, 1)))
    model.add(Flatten())
    model.add(Dense(100, activation="elu"))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation="elu"))
    model.add(Dropout(0.5))
    model.add(Dense(10))
    model.add(Dense(1))
    return model
  
def comma_ai():
    model = Sequential()
    model.add(Cropping2D(cropping=((70, 25), (0, 0)), input_shape=dataset.image_shape))
    model.add(Lambda(lambda x: x/127.5 - 1.0))
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))
    return model
  
model = nvidia_sequential()
model.compile(optimizer="adam", loss="mse")
model.fit_generator(generator=dataset.for_training(),
                    steps_per_epoch=dataset.train_size / dataset.batch_size,
                    validation_data=dataset.for_testing(),
                    validation_steps=dataset.test_size / dataset.batch_size,
                    epochs=3)

model.save('./CarND-Behavioral-Cloning-P3/model.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2), padding="valid")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2), padding="valid")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2), padding="valid")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", strides=(1, 1), padding="valid")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", strides=(1, 1), padding="valid")`


Epoch 1/3
101/100 [==============================] - 112s 1s/step - loss: 0.0282 - val_loss: 0.0235
Epoch 2/3
101/100 [==============================] - 102s 1s/step - loss: 0.0233 - val_loss: 0.0220
Epoch 3/3
101/100 [==============================] - 102s 1s/step - loss: 0.0221 - val_loss: 0.0215


In [56]:
from keras.models import load_model
model = load_model('./CarND-Behavioral-Cloning-P3/model.h5')
model.fit_generator(generator=dataset.for_training(),
                    steps_per_epoch=dataset.train_size / dataset.batch_size,
                    validation_data=dataset.for_testing(),
                    validation_steps=dataset.test_size / dataset.batch_size,
                    epochs=3)



Epoch 1/3
101/100 [==============================] - 103s 1s/step - loss: 0.0308 - val_loss: 0.0259
Epoch 2/3
101/100 [==============================] - 97s 961ms/step - loss: 0.0290 - val_loss: 0.0253
Epoch 3/3
101/100 [==============================] - 97s 963ms/step - loss: 0.0274 - val_loss: 0.0245


In [0]:
# DOWNLOAD TRAINED MODEL FROM GOOGLE COLAB
model.save('./CarND-Behavioral-Cloning-P3/model.h5')
from google.colab import files
files.download('./CarND-Behavioral-Cloning-P3/model.h5')